# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [42]:
!cat /Users/bfetahu/Downloads/citations.tsv

























































DBLP:series/lncs/TaibiFDF16	"Taibi, Davide; Fulantelli, Giovanni; Dietze, Stefan; Fetahu, Besnik"	Educational Linked Data on the Web - Exploring and Analysing the Scope and Coverage		9500			16--37	2016	https://doi.org/10.1007/978-3-319-30493-9\_2	Open Data for Education - Linked, Shared, and Reusable Data for Teaching and Learning			Lecture Notes in Computer Science	"Mouromtsev, Dmitry; d'Aquin, Mathieu"	Springer																									

## Import pandas

We are using the very handy pandas library for dataframes.

In [43]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [113]:
publications = pd.read_csv("/Users/bfetahu/Downloads/citations.tsv", sep="\t", header=0)
publications.head()


,BibliographyType,ISBN,Identifier,Author,Title,Journal,Volume,Number,Month,Pages,...,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44
0,6,NaN,DBLP:conf/bigdataconf/IosifidisFN19,"Iosifidis, Vasileios; Fetahu, Besnik; Ntoutsi,...",FAE: A Fairness-Aware Ensemble Framework,NaN,NaN,NaN,NaN,1375--1380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,DBLP:conf/chi/HubeFG19,"Hube, Christoph; Fetahu, Besnik; Gadiraju, Ujwal",Understanding and Mitigating Worker Biases in ...,NaN,NaN,NaN,NaN,407,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,NaN,DBLP:conf/cikm/FetahuMA15,"Fetahu, Besnik; Markert, Katja; Anand, Avishek",Automated News Suggestions for Populating Wiki...,NaN,NaN,NaN,NaN,323--332,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,NaN,DBLP:conf/cikm/FetahuMNA16,"Fetahu, Besnik; Markert, Katja; Nejdl, Wolfgan...",Finding News Citations for Wikipedia,NaN,NaN,NaN,NaN,337--346,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,NaN,DBLP:conf/dexa/NunesCCFLD13,"Nunes, Bernardo Pereira; Caraballo, Alexander ...",Complex Matching of RDF Datatype Properties,NaN,8055,NaN,NaN,195--208,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [100]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [115]:
import os,re 
for row, item in publications.iterrows():
    pub_date = str(item.Year)
    key_= str(item.Identifier)
    key_ = re.sub(':', '_', re.sub('/', '_', key_))
    title = item.Title
    venue = item.Booktitle
    if len(str(venue)) == 0 or str(venue) == 'nan':
        continue
    
    if len(str(key_)) == 0:
        continue
    md_filename = str(pub_date) + '-' + key_ + ".md"
    html_filename = str(pub_date) + "-" + key_
    year = pub_date[:4]
    
    ## YAML variables
    venue = str(venue)
    venue = re.sub(r'\{|\}', '', venue)
    venue = re.sub(r'\\&',' & ', venue)
    
    md = "---\ntitle: \""   + title + '"\n'
    md += "authors:\"" + item.Author + "\"\n"
    md += """collection: publications"""
    md += """\npermalink: /publication/""" + html_filename
    
    md += "\ndate: " + str(pub_date) + '-01-01'
    md += "\nvenue: \"" + venue + "\""
    
#     md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [116]:
!ls ../_publications/

2012-DBLP_conf_sigir_FetahuS12.md
2013-DBLP_conf_dexa_NunesCCFLD13.md
2013-DBLP_conf_ectel_TaibiFDF13.md
2013-DBLP_conf_esws_NunesDCKFN13.md
2013-DBLP_conf_icwe_FetahuND13.md
2013-DBLP_conf_kes_NunesKFDCM13.md
2013-DBLP_conf_lak_NunesFC13.md
2013-DBLP_conf_semweb_FetahuDNTC13.md
2013-DBLP_conf_semweb_NunesFDC13.md
2013-DBLP_conf_www_FetahuND13.md
2013-DBLP_conf_www_TaibiFD13.md
2014-DBLP_conf_esws_FetahuDNCTN14.md
2014-DBLP_conf_icalt_NunesCKFCC14.md
2014-DBLP_conf_semweb_FetahuGD14.md
2014-DBLP_conf_semweb_TaibiDFF14.md
2014-DBLP_conf_wise_NunesKFCC14.md
2014-DBLP_conf_www_FetahuDNCTN14.md
2015-DBLP_conf_cikm_FetahuMA15.md
2015-DBLP_conf_ectel_GadirajuFK15.md
2015-DBLP_conf_ht_GadirajuSFK15.md
2015-DBLP_conf_semweb_FetahuGD15.md
2015-DBLP_conf_websci_FetahuAA15.md
2015-DBLP_conf_wise_YuGFD15.md
2015-DBLP_conf_www_TaibiFDF15.md
2016-DBLP_conf_cikm_FetahuMNA16.md
2016-DBLP_conf_esws_YuGZFD16.md
2016-DBLP_conf_semweb_YuFGD16.md
2016-DBLP_series_lncs_TaibiFDF16.md
2017-DBLP_conf_ectel_dAq

In [117]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
